# Notebook for project results in 02456 Deep Learning
This notebook will load our final model and will perform inference on 6 randomly selected clean-noise pairs from the test set.

Firstly, the model and audio files will be downloaded from Google Drive. Note that these will be stored locally in the folder "notebook_files".

In [43]:
import os
import gdown

# Create notebook folder for temporary files
directory = "notebook_files/"
if not os.path.exists(directory):
    os.makedirs(directory)

url = "https://drive.google.com/drive/folders/1jRPZWRAQE2PKvnc5zpxK1r09J7OZjWsR?usp=drive_link"
gdown.download_folder(url,output=directory, quiet=True, use_cookies=False)
1

Access denied with the following error:



 	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses. 

You may still be able to access the file from the browser:

	 https://drive.google.com/uc?id=1lSWdYqXM4VbwwXGg8Xj0qVxKvPYCLjoq 



1

## Loading the model


In [40]:
import torch
from models.autoencoder_without_PQC.AudioDec import Generator as GeneratorAudioDec

config = {
    "input_channels": 1,
    "output_channels": 1,
    "encode_channels": 32,
    "decode_channels": 32,
    "bias": True,
    "enc_ratios": [2, 4, 8, 16],
    "dec_ratios": [16, 8, 4, 2],
    "enc_strides": [3, 4, 5, 5],
    "dec_strides": [5, 5, 4, 3],
    "mode": 'causal'
}

model: dict[str, torch.nn.Module] = {}
model["generator"] = GeneratorAudioDec(**config)
state_dict = torch.load("notebook_files/model.pkl", map_location="cpu")
model["generator"].load_state_dict(state_dict)

SAMPLE_RATE = 24000


<All keys matched successfully>

## Load data


In [63]:
import torchaudio
import glob
from dataloader.data_utils import add_noise


def get_file_paths(prefix):
    return glob.glob(directory+prefix+"*"+".wav")
clean_file_paths = get_file_paths("clean")
noise_file_paths = get_file_paths("noise")

def load_samples(file_paths):
    samples = []
    for file_path in file_paths:
        audio, _ = torchaudio.load(file_path, backend="soundfile")
        samples.append(audio)
    return  samples


clean_samples = load_samples(clean_file_paths)
noise_samples = load_samples(noise_file_paths)

# TODO - Down-sample

for i, (clean_sample,noise_sample) in enumerate(zip(clean_samples,noise_samples)):
    if clean_sample.shape[1] > noise_sample.shape[1]:
        clean_samples[i] = clean_sample[:, :noise_sample.shape[1]]
    else:
        noise_samples[i] = noise_sample[:, :clean_sample.shape[1]]

mixed_samples = [add_noise(speech,noise,torch.tensor(15)) for speech,noise in zip(clean_samples,noise_samples)]




In [61]:
import torch

# Create a dummy tensor of shape (1, 1500)
tensor = torch.rand(1, 1500)

# Trim the tensor to shape (1, 1000)
trimmed_tensor = tensor[:, :1000]

print("Original shape:", tensor.shape)
print("Trimmed shape:", trimmed_tensor.shape)

Original shape: torch.Size([1, 1500])
Trimmed shape: torch.Size([1, 1000])
